In [67]:
import numpy as np
import torch
F = torch.nn.functional
nn = torch.nn

#5 classes
n=20
c=5 # n classes
f=10
h=8
torch.manual_seed(0)

def cmp(a,b):
    assert a.shape==b.shape
    assert torch.isclose(a,b.grad).all(), f'{a}\n{b.grad}'

X = torch.randn(n, f)
y = torch.randint(low=0,high=5,size=(n,))

In [68]:
w1 = torch.randn(f, h) * np.sqrt(1/f)
b1 = torch.zeros(h)
w2 = torch.randn(h, h) * np.sqrt(1/h)
b2 = torch.zeros(h)
w3 = torch.randn(h, h) * np.sqrt(1/h)
b3 = torch.zeros(h)
w4 = torch.randn(h, c) * np.sqrt(1/h)
b4 = torch.zeros(c)

params = [w1,b1,w2,b2,w3,b3,w4,b4]
for p in params:
    p.requires_grad=True

In [69]:
# # forward
# def forward(x):
#     x2 = x@w1+b1
#     x2act = torch.maximum(x2, torch.zeros_like(x2))
#     x3 = x2act@w2+b2
#     x3act = torch.maximum(x3, torch.zeros_like(x3))
#     x4 = x3act@w3+b3
#     x4act = torch.maximum(x4, torch.zeros_like(x4))
#     logits = x4act@w4+b4
    
#     logits2 = logits - logits.max(1,keepdims=True).values
#     probas = torch.exp(logits2) / torch.exp(logits2).sum(1,keepdims=True)
#     log_probs = torch.log(probas[range(n), y])
#     loss = -log_probs.sum() / n
    
#     for v in [x2,x2act,x3,x3act,x4,x4act,logits,logits2,probas,log_probs]:
#         v.retain_grad()
        
#     return probas, loss

In [70]:
# def backward():
#     dlogits = probas.clone()
#     dlogits[range(n), y] -= 1
#     dlogits /= n
    
#     dx4act = dlogits@w4.T
#     dw4 = x4act.T@dlogits
#     dx4 = dx4act.clone()
#     dx4[x4<0] = 0.0
#     db4 = dlogits.sum(0)
    
#     dx3act = dx4@w3.T
#     dw3 = x3act.T@dx4
#     dx3 = dx3act.clone()
#     dx3[x3<0] = 0.0
#     db3 = dx4.sum(0)
    
#     dx2act = dx3@w2.T
#     dw2 = x2act.T@dx3
#     dx2 = dx2act.clone()
#     dx2[x2<0] = 0.0
#     db2 = dx3.sum(0)
    
#     # dx1 = dx2@w1.T
#     dw1 = x1.T@dx2
#     db1 = dx2.sum(0)
    
#     # cmp(dlogits, logits)
#     # cmp(dx4act, x4act)
#     # cmp(dw4, w4)
#     # cmp(dx4, x4)
#     # cmp(db4, b4)
    
#     # cmp(dx3act, x3act)
#     # cmp(dw3, w3)
#     # cmp(dx3, x3)
#     # cmp(db3, b3)
    
    
#     # cmp(dx2act, x2act)
#     # cmp(dw2, w2)
#     # cmp(dx2, x2)
#     # cmp(db2, b2)
    
#     # cmp(dw1, w1)
#     # # cmp(dx1, x1)
#     # cmp(db1, b1)

#     return [dw1,db1,dw2,db2,dw3,db3,dw4,db4]

In [95]:
lr = .1

for i in range(1):
    # FORWARD
    x2 = X@w1+b1
    x2act = torch.maximum(x2, torch.zeros_like(x2))
    x3 = x2act@w2+b2
    x3act = torch.maximum(x3, torch.zeros_like(x3))
    x4 = x3act@w3+b3
    x4act = torch.maximum(x4, torch.zeros_like(x4))
    logits = x4act@w4+b4
    
    logits2 = logits - logits.max(1,keepdims=True).values
    probas = torch.exp(logits2) / torch.exp(logits2).sum(1,keepdims=True)
    log_probs = torch.log(probas)
    loss = -log_probs[range(n), y].sum() / n
    
    # for v in [x2,x2act,x3,x3act,x4,x4act,logits,logits2,probas,log_probs]:
    #     v.retain_grad()

    acc = (torch.argmax(probas,1) == y).to(torch.float).mean()
    print(f'loss: {loss.detach().item():.5f} acc:{acc:.4}')

    # BACKWARD
    dlogits = probas.clone()
    dlogits[range(n), y] -= 1
    dlogits /= n
    
    dx4act = dlogits@w4.T
    dw4 = x4act.T@dlogits
    dx4 = dx4act.clone()
    dx4[x4<0] = 0.0
    db4 = dlogits.sum(0)
    
    dx3act = dx4@w3.T
    dw3 = x3act.T@dx4
    dx3 = dx3act.clone()
    dx3[x3<0] = 0.0
    db3 = dx4.sum(0)
    
    dx2act = dx3@w2.T
    dw2 = x2act.T@dx3
    dx2 = dx2act.clone()
    dx2[x2<0] = 0.0
    db2 = dx3.sum(0)
    
    # dx1 = dx2@w1.T
    dw1 = X.T@dx2
    db1 = dx2.sum(0)

    dparams = [dw1,db1,dw2,db2,dw3,db3,dw4,db4]
    
    # for p in params:
    #     p.grad = None
    # loss.backward() # comment later
    # dparams = backward()
    
    for dp,p in zip(dparams, params):
        assert dp.shape == p.shape
        with torch.no_grad():
            p -= dp * lr

loss: 1.12002 acc:0.55


In [140]:

# def check(d, x):
    

# dlogprobs

# dlogits = (y_proba-1) for correct class, y_proba for incorrect class

dlog_probs = np.ones_like(log_probs) * -1/n
dprobas = 1.0/probas * dlog_probs
dlogits_exp_true = dprobas * 1/logits_sum
dlogits_sum = 1/logits_exp_true * dprobas
dlogits_exp = np.zeros_like(logits_exp)
dlogits_exp[range(n), y] = dlogits_exp_true
dlogits_exp = dlogits_sum[:, None] * np.ones_like(logits_exp)
dlogits2 = np.exp(logits2)
dlogits = dlogits2
dlogits_max = -dlogits2.sum(1,keepdims=True)
dw4 = dlogits.T@x3
assert dw4.shape == w4.shape
dx4 = x3.T@dlogits
dw3 = dx4.T@w3
dx3 = w3.T@dx4
dw2 = dx3.T@x2
dx2 = x2.T@dx3
dw2 = dx2.T@w1
dx1 = w1.T@dx2


logits_exp_true
    
    
    

loss = forward(X)

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.